In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,StandardScaler
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 50)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
class path :
    app_open='app_open.csv'
    given_clients='given_clients.csv'
    market_order='market_order.csv'
#     read_me='/kaggle/input/getir-case/getir/read_me.pdf'
    product_detail='product_detail.csv'
    promo_detail='promo_detail.csv'
    

> ### given_clients
- Buradaki clinet lar tahmin etmemiz gerekn new user lar


In [2]:
given_clients=pd.read_csv(path.given_clients)
given_clients

,client_id
0,6538ff945bbb65f457cc0150
1,6538ff945bbb65f457cc10e9
2,6538ff945bbb65f457cc162b
3,6538ff945bbb65f457cc17eb
4,6538ff945bbb65f457cc1b17
...,...
27561,6538ff945bbb65f457ccbfd5
27562,6538ff945bbb65f457ccbffa
27563,6538ff945bbb65f457cc9d1a
27564,6538ff945bbb65f457cc9d2d


> #### >  app_open

In [3]:
app_open=pd.read_csv(path.app_open)
# app_open.appopendate=pd.to_datetime(app_open.appopendate)

print(f'app_open unique client sayisi :{app_open.client_id.nunique()}')
print(f'app_open max date  :{app_open.appopendate.max()}')
print(f'app_open min date :{app_open.appopendate.min()}')

# app_open.appopendate=app_open.appopendate.dt.date
app_open.appopendate=pd.to_datetime(app_open.appopendate)
app_open.appopendate=app_open.appopendate.dt.strftime("%Y-%m-%d-%H:%M")
app_open.appopendate=pd.to_datetime(app_open.appopendate)

app_open=app_open.drop_duplicates()
print(f'app_open shape :{app_open.shape}')
#kullanicin app a girme sıklığı 
dum_app_open=app_open.groupby('client_id').agg(app_open_date_max=('appopendate', 'max'),
                                    app_open_date_min=('appopendate', 'min'),
                                    appopendate_count=('appopendate', 'count'),
                                    date_range_hour=('appopendate', lambda x: (x.max() - x.min()).total_seconds()/3600))

dum_app_open['app_kullanma_freq']=dum_app_open.appopendate_count/dum_app_open.date_range_hour
dum_app_open.replace(np.inf,0,inplace=True)

app_open=app_open.merge(dum_app_open,on='client_id',how='left')
app_open=app_open[['client_id','appopendate_count','app_kullanma_freq']].drop_duplicates()
app_open

app_open unique client sayisi :192822
app_open max date  :2021-01-27 23:59:58.518
app_open min date :2020-06-30 00:00:23.420
app_open shape :(5075425, 2)


,client_id,appopendate_count,app_kullanma_freq
0,6538ff945bbb65f457caae91,90,0.021499
1,6538ff945bbb65f457ca7ba0,44,0.008838
2,6538ff945bbb65f457cb17f1,31,0.006753
3,6538ff945bbb65f457ca6df8,26,0.005238
4,6538ff945bbb65f457ca9358,72,0.014771
...,...,...,...
5072340,6538ff945bbb65f457cd486d,1,0.000000
5074279,6538ff945bbb65f457cd2e4c,1,0.000000
5074642,6538ff945bbb65f457ccfc98,1,0.000000
5075048,6538ff945bbb65f457cd3a34,1,0.000000


> ### product_detail

In [4]:
product_detail=pd.read_csv(path.product_detail)
print(f'product_detail unique order_id sayisi :{product_detail.order_id.nunique()}')
product_detail.drop('_pos',axis=1,inplace=True)

product_detail['count_price']=product_detail['count']*product_detail['price']
dum_product_detail = product_detail.groupby('order_id')['count_price'].agg(count_price_sum='sum', count_price_mean='mean')

count_product_detail = product_detail.groupby('order_id')['count'].agg(count_sum='sum')
product_detail=product_detail.merge(count_product_detail,on='order_id',how='left')

product_detail=product_detail.merge(dum_product_detail,on='order_id',how='left')
product_detail.drop('count_price',axis=1,inplace=True)


product_detail_exploded=product_detail.copy()
product_detail_exploded['count']=product_detail_exploded['count'].apply(lambda x: [x] * x)
product_detail_exploded = product_detail_exploded.explode('count')
# -----
# mode of mastercategory,subcategory,mode_brand,mode_product  
mode_mastercategory=product_detail_exploded.groupby('order_id').agg({'mastercategory_id': lambda x: x.mode()[0]}).rename(columns={'mastercategory_id':'mastercategory_mode'})
mode_subcategory=product_detail_exploded.groupby('order_id').agg({'subcategory_id': lambda x: x.mode()[0]}).rename(columns={'subcategory_id':'subcategory_mode'})
mode_brand=product_detail_exploded.groupby('order_id').agg({'brand_id': lambda x: x.mode()[0]}).rename(columns={'brand_id':'brand_mode'})
mode_product=product_detail_exploded.groupby('order_id').agg({'product_id': lambda x: x.mode()[0]}).rename(columns={'product_id':'product_mode'})

# ----
product_detail=product_detail.merge(pd.concat([mode_mastercategory,mode_subcategory,mode_brand,mode_product],axis=1),on='order_id',how='left')
product_detail.drop(['subcategory_id','brand_id','mastercategory_id','count','product_id','price'],axis=1,inplace=True)
product_detail=product_detail.drop_duplicates()
product_detail

product_detail unique order_id sayisi :444825


,order_id,count_sum,count_price_sum,count_price_mean,mastercategory_mode,subcategory_mode,brand_mode,product_mode
0,6539005e5bbb65f457d50370,19,51.7604,10.35208,653902515bbb65f457de3ba2,653902045bbb65f457de3677,653902215bbb65f457de397f,653901c45bbb65f457de33b6
1,6539005f5bbb65f457d6882e,2,23.5900,11.79500,653902515bbb65f457de3b9d,653902045bbb65f457de36a5,653902215bbb65f457de3ad9,653901c45bbb65f457de3287
2,6539005f5bbb65f457d68829,2,23.5900,11.79500,653902515bbb65f457de3b9d,653902045bbb65f457de377a,653902215bbb65f457de3ad9,653901c45bbb65f457de3287
3,6539005f5bbb65f457d63a77,4,24.8600,12.43000,653902515bbb65f457de3b9d,653902045bbb65f457de36a5,653902215bbb65f457de3923,653901c45bbb65f457de2cc3
4,6539005e5bbb65f457d4e4d4,4,20.7400,5.18500,653902515bbb65f457de3b9d,653902045bbb65f457de36a0,653902215bbb65f457de3908,653901c45bbb65f457de2c9f
...,...,...,...,...,...,...,...,...
2550096,6539005d5bbb65f457d14b4b,10,20.9000,20.90000,653902515bbb65f457de3bd6,653902045bbb65f457de3862,653902215bbb65f457de3a4b,653901c45bbb65f457de32f9
2550178,6539005e5bbb65f457d223cb,7,14.6300,14.63000,653902515bbb65f457de3bd6,653902045bbb65f457de3862,653902215bbb65f457de3a4b,653901c45bbb65f457de32f9
2550272,6539005d5bbb65f457d19511,7,13.9300,13.93000,653902515bbb65f457de3bd6,653902045bbb65f457de3862,653902215bbb65f457de3a4b,653901c45bbb65f457de32f9
2550389,6539005d5bbb65f457d0e7b7,6,12.5400,12.54000,653902515bbb65f457de3bd6,653902045bbb65f457de3862,653902215bbb65f457de3a4b,653901c45bbb65f457de32f9


> ### promo_detail

In [6]:
promo_detail=pd.read_csv(path.promo_detail)
promo_detail.promoobjective=promo_detail.promoobjective.astype(int)
print(f'product_detail unique order_id sayisi :{promo_detail.order_id.nunique()}')
promo_detail

product_detail unique order_id sayisi :315118


,order_id,promo_id,responsibledepartment_id,promoobjective
0,6539005e5bbb65f457d50b5f,653903915bbb65f457de3bdc,653903b15bbb65f457de3fe3,2
1,6539005e5bbb65f457d4c74a,653903915bbb65f457de3bdc,653903b15bbb65f457de3fe3,2
2,6539005e5bbb65f457d41b53,653903915bbb65f457de3bdc,653903b15bbb65f457de3fe3,2
3,6539005e5bbb65f457d21191,653903915bbb65f457de3bdc,653903b15bbb65f457de3fe3,2
4,6539005e5bbb65f457d277d7,653903915bbb65f457de3bdc,653903b15bbb65f457de3fe3,2
...,...,...,...,...
315113,6539005e5bbb65f457d54d45,653903915bbb65f457de3fde,653903b15bbb65f457de3fe3,5
315114,6539005f5bbb65f457d6a1d2,653903915bbb65f457de3fdf,653903b15bbb65f457de3fe4,9
315115,6539005f5bbb65f457d7162d,653903915bbb65f457de3fe0,653903b15bbb65f457de3fe4,9
315116,6539005f5bbb65f457d70218,653903915bbb65f457de3fe1,653903b15bbb65f457de3fe3,2


> #### product_detail_promo_detail merged

In [7]:
product_detail_promo_detail=product_detail.merge(promo_detail,on='order_id',how='left')
product_detail_promo_detail.fillna('no_promo',inplace=True)
product_detail_promo_detail.promoobjective.replace('no_promo',0,inplace=True)
product_detail_promo_detail

,order_id,count_sum,count_price_sum,count_price_mean,mastercategory_mode,subcategory_mode,brand_mode,product_mode,promo_id,responsibledepartment_id,promoobjective
0,6539005e5bbb65f457d50370,19,51.7604,10.35208,653902515bbb65f457de3ba2,653902045bbb65f457de3677,653902215bbb65f457de397f,653901c45bbb65f457de33b6,653903915bbb65f457de3caa,653903b15bbb65f457de3fe3,4.0
1,6539005f5bbb65f457d6882e,2,23.5900,11.79500,653902515bbb65f457de3b9d,653902045bbb65f457de36a5,653902215bbb65f457de3ad9,653901c45bbb65f457de3287,no_promo,no_promo,0.0
2,6539005f5bbb65f457d68829,2,23.5900,11.79500,653902515bbb65f457de3b9d,653902045bbb65f457de377a,653902215bbb65f457de3ad9,653901c45bbb65f457de3287,no_promo,no_promo,0.0
3,6539005f5bbb65f457d63a77,4,24.8600,12.43000,653902515bbb65f457de3b9d,653902045bbb65f457de36a5,653902215bbb65f457de3923,653901c45bbb65f457de2cc3,no_promo,no_promo,0.0
4,6539005e5bbb65f457d4e4d4,4,20.7400,5.18500,653902515bbb65f457de3b9d,653902045bbb65f457de36a0,653902215bbb65f457de3908,653901c45bbb65f457de2c9f,653903915bbb65f457de3f1c,653903b15bbb65f457de3fe3,1.0
...,...,...,...,...,...,...,...,...,...,...,...
444820,6539005d5bbb65f457d14b4b,10,20.9000,20.90000,653902515bbb65f457de3bd6,653902045bbb65f457de3862,653902215bbb65f457de3a4b,653901c45bbb65f457de32f9,653903915bbb65f457de3f1c,653903b15bbb65f457de3fe3,1.0
444821,6539005e5bbb65f457d223cb,7,14.6300,14.63000,653902515bbb65f457de3bd6,653902045bbb65f457de3862,653902215bbb65f457de3a4b,653901c45bbb65f457de32f9,653903915bbb65f457de3d7f,653903b15bbb65f457de3fe3,1.0
444822,6539005d5bbb65f457d19511,7,13.9300,13.93000,653902515bbb65f457de3bd6,653902045bbb65f457de3862,653902215bbb65f457de3a4b,653901c45bbb65f457de32f9,653903915bbb65f457de3f39,653903b15bbb65f457de3fe3,2.0
444823,6539005d5bbb65f457d0e7b7,6,12.5400,12.54000,653902515bbb65f457de3bd6,653902045bbb65f457de3862,653902215bbb65f457de3a4b,653901c45bbb65f457de32f9,653903915bbb65f457de3cc2,653903b15bbb65f457de3fe3,2.0


> ### market_order

In [100]:
market_order=pd.read_csv(path.market_order,low_memory=False)
market_order.orderdate=pd.to_datetime(market_order.orderdate)
market_order.createdat=pd.to_datetime(market_order.createdat)
market_order.deliverdate=pd.to_datetime(market_order.deliverdate)
market_order.drop(['ratereason','rating','ratedate','rateisskipped','thirdpartysupport'],axis=1,inplace=True)

print(f'market_order shape: {market_order.shape}')
print(f'orderdate min    : {market_order.orderdate.min()} -- orderdate max   : {market_order.orderdate.max()}')
print(f'createdat min    : {market_order.createdat.min()} -- createdat max   : {market_order.createdat.max()}')
print(f'deliverdate min  : {market_order.deliverdate.min()} -- deliverdate max : {market_order.deliverdate.max()}')
print(f'market_order tablosu ,client_id  unique. sayısı : {market_order.client_id.nunique()}')
print(f'market_order tablosu ,order_id  unique. sayısı  : {market_order.order_id.nunique()}')

#device name 
market_order.devicename=list(map(lambda x: x[1] if len(x) > 1 else x, market_order['devicename'].str.split(';')))
market_order.devicename=market_order.devicename.str.lower()
market_order.loc[~market_order.devicename.isin(market_order.devicename.value_counts()[:13].index),'devicename']='unknown'


#kullanici kac farkli adres için sipariş veriyor?
# city_sum=market_order.groupby('client_id')['city_id'].agg(city_sum='nunique')
addressname_unique=market_order.groupby('client_id')['addressname'].agg(addressname_unique='nunique')

#kullanici bazli delivery ulaştırılma süresi ortalması ve kullanıcın order ı oluşturma süresinin ortalması
market_order['delivery_order_date_diff_minute']=list(map(lambda x: x.total_seconds() / 60,(market_order.deliverdate-market_order.orderdate)))
market_order['order_created_date_diff_minute']=list(map(lambda x: x.total_seconds() / 60,(market_order.orderdate-market_order.createdat)))

delivery_order_date_diff_minute_mean=market_order.groupby('client_id')['delivery_order_date_diff_minute'].agg(delivery_order_date_diff_minute_mean='mean')
order_created_date_diff_minute_mean=market_order.groupby('client_id')['order_created_date_diff_minute'].agg(order_created_date_diff_minute_mean='mean')

# orderdaki toplam indirim oranı
market_order['order_indirim_oran']=market_order.discountamount/market_order.basketvalue
market_order['order_indirim_oran'].replace(np.inf,0,inplace=True)
order_indirim_oran_by_client_sum = market_order.groupby('client_id').agg(order_indirim_oran_by_client_sum=('order_indirim_oran','sum'))

# müşteri  order sıklığı 
order_count = market_order.groupby('client_id').agg(order_count=('orderdate', 'count'))
order_date_range_day_by_client= market_order.groupby('client_id').agg(order_date_range_by_client_day=('orderdate', lambda x: (x.max() - x.min()).total_seconds()/(3600*24)))
order_feq=order_count.order_count/order_date_range_day_by_client.order_date_range_by_client_day
order_feq.replace(np.inf,0.000001,inplace=True)



dum_concat=pd.concat([addressname_unique,
                      delivery_order_date_diff_minute_mean,
                      order_created_date_diff_minute_mean,
                      order_indirim_oran_by_client_sum,
                      order_count,
                      order_date_range_day_by_client,
                      order_feq],axis=1)
market_order=market_order.merge(dum_concat,on='client_id',how='left')

# warehouse ların ortalma order telsim etme süreleri saatlik
warehouse_delivery_mean = market_order.groupby('warehouse_id').agg(order_count=('warehouse_delivery_mean', 'mean'))
market_order.merge(warehouse_delivery_mean,on='warehouse_id',how='left')


market_order.drop(['city_id','hex_id','addressname'],axis=1,inplace=True)
market_order

market_order shape: (444825, 17)
orderdate min    : 2020-06-30 06:11:14.317000 -- orderdate max   : 2021-01-27 23:53:47.139000
createdat min    : 2020-06-29 07:23:31.283000 -- createdat max   : 2021-01-27 23:53:46.491000
deliverdate min  : 2020-06-30 06:21:28.509000 -- deliverdate max : 2021-01-28 00:02:52.422000
market_order tablosu ,client_id  unique. sayısı : 198214
market_order tablosu ,order_id  unique. sayısı  : 444825


KeyError: "Column(s) ['warehouse_delivery_mean'] do not exist"

In [ ]:
df_train=market_order.merge(app_open,on='client_id',how='left').merge(product_detail_promo_detail,on='order_id',how='left')

In [ ]:
result = df_train.groupby('client_id').agg(
order_count=('order_id', 'count'),
orderdate_max=('orderdate', 'max'),
orderdate_min=('orderdate', 'min'),    
deliverdate_max=('deliverdate', 'max'),
deliverdate_min=('deliverdate', 'min'),
createdat_max=('createdat', 'max'),
createdat_min=('createdat', 'min'),
devicename=('devicename', lambda x: x.mode()[0]),
# addressname=('addressname', lambda x: x.mode()[0]),
unique_addressname=('addressname_unique',  lambda x :x ),
distance_mean=('distance', 'mean'),
andm_median=('aandm', 'median'),
basketvalue_sum=('basketvalue', 'sum'),
chargedamount_sum=('chargedamount', 'sum'),
deliveryfee_mean=('deliveryfee', 'sum'),
discountamount_mean=('discountamount', 'sum'),
supliersupport_mean=('supliersupport', 'sum'),
warehouse_id_mode=('warehouse_id', lambda x: x.mode()[0]),
delivery_order_date_diff_minute_mean=('delivery_order_date_diff_minute_mean', lambda x :x ),
order_created_date_diff_minute_mean=('order_created_date_diff_minute', lambda x :x ),
toplam_indirim_oran_sum =('toplam_indirim_oran', 'sum'),
count_order_product_sum=('count_sum', 'sum'),
count_price_order_sum=('count_price_sum', 'mean'),
count_price_mean=('count_price_mean', 'mean'),
mastercategory_mode=('mastercategory_mode', lambda x: x.mode()[0]),
subcategory_mode=('subcategory_mode', lambda x: x.mode()[0]),
brand_mode=('brand_mode', lambda x: x.mode()[0]),
product_mode=('product_mode', lambda x: x.mode()[0]),
promo_id=('promo_id', lambda x: x.mode()[0]),
responsibledepartment_id=('responsibledepartment_id', lambda x: x.mode()[0]),
promoobjective=('promoobjective', lambda x: x.mode()[0]))

df=result.copy()
df['ilkveson_order_date_diff']=list(map(lambda x: round(x.total_seconds()/(3600),2),(df.orderdate_max-df.orderdate_min)))
df.drop(['orderdate_max','orderdate_min','deliverdate_max','deliverdate_min','createdat_max','createdat_min'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
given_clients_df=df.loc[given_clients.client_id]
no_given_clients_df=df.loc[~df.index.isin(given_clients.client_id)]

In [ ]:
no_given_clients_df.select_dtypes('object').columns

In [ ]:
no_given_clients_df.info()

In [ ]:
def label_data(feature,data,le_fit=None):
    if type(feature)==str:
        feature=[feature]
    else :
        pass
    for col in feature:
        le=LabelEncoder()
        le_fit.update({col:le.fit(data[col])})
        data[col]=le.transform(data[col])
    return data[feature],le_fit

cat_columns=['devicename', 'warehouse_id_mode', 'mastercategory_mode',
           'subcategory_mode', 'brand_mode', 'product_mode', 'promo_id',
           'responsibledepartment_id', 'promoobjective']

no_given_clients_df.loc[:,cat_columns],train_le_list=label_data(cat_columns,no_given_clients_df,le_fit={})
# test.loc[:,cat_columns]=pd.DataFrame([v.transform(test[k]) for k,v in list(train_le_list.items())]).T.values

In [ ]:
no_given_clients_df.info()

In [ ]:
no_given_clients_df['target']=labels

In [ ]:
plt.figure(figsize=(15,20))
# Adjust the space between subplots
plt.subplots_adjust(hspace=1)
plt.subplot(5,2,1);
ax=sns.barplot(x='target',y='basketvalue_sum',data=no_given_clients_df,estimator='sum');
ax.set_title('basketvalue_sum by target');

plt.subplot(5,2,2);
ax=sns.barplot(x='target',y='order_count',data=no_given_clients_df,estimator='sum');
ax.set_title('order_count by target');

plt.subplot(5,2,3);
ax=sns.barplot(x='target',y='deliveryfee_mean',data=no_given_clients_df,estimator='sum');
ax.set_title('deliveryfee_mean by target');

plt.subplot(5,2,4);
ax=sns.barplot(x='target',y='delivery_order_date_diff_minute_mean',data=no_given_clients_df,estimator='sum');
ax.set_title('delivery_order_date_diff_minute_mean by target');

plt.subplot(5,2,5);
ax=sns.barplot(x='target',y='order_created_date_diff_minute_mean',data=no_given_clients_df,estimator='sum');
ax.set_title('order_created_date_diff_minute_mean by target');

plt.subplot(5,2,6);
ax=sns.barplot(x='target',y='toplam_indirim_oran_sum',data=no_given_clients_df,estimator='sum');
ax.set_title('toplam_indirim_oran_sum by target');

plt.subplot(5,2,7);
ax=sns.barplot(x='target',y='count_order_product_sum',data=no_given_clients_df,estimator='sum');
ax.set_title('count_order_product_sum by target')

plt.subplot(5,2,8);
ax=sns.barplot(x='target',y='ilkveson_order_date_diff',data=no_given_clients_df,estimator='sum');
ax.set_title('ilkveson_order_date_diff by target');





In [ ]:
from sklearn.cluster import KMeans
std=StandardScaler()
norm_no_given_clients_df=std.fit_transform(no_given_clients_df)
labels=KMeans(n_clusters=5,random_state=123).fit_predict(norm_no_given_clients_df)   


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Generate random 3D data
# np.random.seed(42)
# data_points = np.random.rand(100, 3)

# Apply k-means clustering with 5 clusters
kmeans =KMeans(n_clusters=5,random_state=123)
labels = kmeans.fit_predict(norm_no_given_clients_df)
centroids = kmeans.cluster_centers_

# Plotting the 3D figure
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for data points with 5 different colors
ax.scatter(norm_no_given_clients_df[:, 0], norm_no_given_clients_df[:, 1], norm_no_given_clients_df[:, 2], c=labels, marker='o', cmap='viridis', s=50)

# Scatter plot for centroids
ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2], c='red', marker='X', s=200, label='Centroids')

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

ax.set_title('K-Means Clustering with 5 Classes in 3D')
ax.legend()

plt.show()


In [ ]:
import plotly.graph_objects as go
# Generate random 3D data
# np.random.seed(42)
# data_points = np.random.rand(100, 3)

# Apply k-means clustering with 5 clusters
kmeans =KMeans(n_clusters=5,random_state=123)
labels = kmeans.fit_predict(norm_no_given_clients_df)
centroids = kmeans.cluster_centers_

# Create an interactive 3D scatter plot
fig = go.Figure()

# Scatter plot for data points with 5 different colors
scatter = fig.add_trace(go.Scatter3d(
    x=norm_no_given_clients_df[:, 0],
    y=norm_no_given_clients_df[:, 1],
    z=norm_no_given_clients_df[:, 2],
    mode='markers',
    marker=dict(color=labels, size=5, colorscale='Viridis'),
    name='Data Points'
))

# Scatter plot for centroids
scatter_centroids = fig.add_trace(go.Scatter3d(
    x=centroids[:, 0],
    y=centroids[:, 1],
    z=centroids[:, 2],
    mode='markers',
    marker=dict(color='red', size=10, symbol='x'),
    name='Centroids'
))

# Layout settings
fig.update_layout(
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    title='Interactive K-Means Clustering',
    showlegend=True
)

# Show the interactive plot
fig.show()


> ### given_clients tablosundaki clientların market order datası

In [ ]:
dum1=df_train[df_train.client_id.isin(given_clients.client_id)]
print(f'dum1 shape: {dum1.shape}')
print(f'orderdate min    : {dum1.orderdate.min()} -- orderdate max   : {dum1.orderdate.max()}')
# print(f'createdat min    : {dum1.createdat.min()} -- createdat max   : {dum1.createdat.max()}')
# print(f'deliverdate min  : {dum1.deliverdate.min()} -- deliverdate max : {dum1.deliverdate.max()}')

In [ ]:
dum1

> ### given_clients tablosundaki client_id'ler haricindeki clinet_id lerin  market order datası

In [ ]:
dum2=df_train[~df_train.client_id.isin(given_clients.client_id)]
print(f'dum2 shape: {dum2.shape}')
print(f'orderdate min    : {dum2.orderdate.min()} -- orderdate max   : {dum2.orderdate.max()}')
# print(f'createdat min    : {dum2.createdat.min()} -- createdat max   : {dum2.createdat.max()}')
# print(f'deliverdate min  : {dum2.deliverdate.min()} -- deliverdate max : {dum2.deliverdate.max()}')

In [ ]:
dum2a=dum2[dum2.orderdate<'2020-10-30 07:11:45.199000']
dum2b=dum2[dum2.orderdate>'2020-10-30 07:11:45.199000']
print(f'dum2a shape: {dum2.shape}')
print(f'dum2a shape: {dum2a.shape}')
print(f'dum2b shape: {dum2b.shape}')

In [ ]:
# churn etmemiş clienlar
# 2020-10-30 öcesinde ve sonrasında olan clinetlar
len(list(set(dum2a.client_id.unique()) & set(dum2b.client_id.unique())))
dumös=list(set(dum2a.client_id.unique()) & set(dum2b.client_id.unique()))

In [ ]:
df_train[df_train.client_id.eq('6538ff945bbb65f457caa56e')]

In [ ]:
dumös

In [ ]:
# dum2a da olan dum2b de olmayan clientlar
# 2020-10-30 öncesinde olup sonrasında olmayan clinetlar
len(list(set(dum2a['client_id'].unique()) - set(dum2b['client_id'].unique())))

In [ ]:
# dum2a da olan dum2b de olmayan clientlar
# 2020-10-30 sonrasında olup öncesinde olmayan clinetlar
# market order tablosunda olan bu kayıları sildim 
len(list(set(dum2b.client_id.unique())-set(dum2a.client_id.unique())))

In [ ]:
df=df_train[~df_train.client_id.isin(list(set(dum2b.client_id.unique())-set(dum2a.client_id.unique())))]

In [ ]:
df.shape

In [ ]:
df.loc[df.client_id.isin(list(set(dum2a.client_id.unique()) & set(dum2b.client_id.unique()))),'target']

In [ ]:
df.loc[df.client_id.isin(list(set(dum2a['client_id'].unique()) - set(dum2b['client_id'].unique()))),'target']

In [ ]:
df['target']=np.nan
df.loc[df.client_id.isin(list(set(dum2a.client_id.unique()) & set(dum2b.client_id.unique()))),'target']=0
df.loc[df.client_id.isin(list(set(dum2a['client_id'].unique()) - set(dum2b['client_id'].unique()))),'target']=1

In [ ]:
df.isnull().sum()

In [ ]:
df_train[df_train.client_id.eq('6538ff945bbb65f457cc64de')]

In [ ]:
# İki tarih aralığı
dum1_orderdate_min = dum1.orderdate.min()
dum1_orderdate_max = dum1.orderdate.max()
dum2_orderdate_min = dum2.orderdate.min()
dum2_orderdate_max = dum2.orderdate.max()
# İki tarih aralığının kesişim tarihleri
intersection_min = max(dum1_orderdate_min, dum2_orderdate_min)
intersection_max = min(dum1_orderdate_max, dum2_orderdate_max)
# Grafik çizimi
plt.figure(figsize=(15, 5))
plt.plot([intersection_min, intersection_max], [0, 0], marker='o', label='Kesişim')
# İlk tarih aralığı
plt.plot([dum1_orderdate_min, dum1_orderdate_max], [1, 1], marker='o', label='Given clients tablosundaki clinet larin max ve min tarih aralıkları')
# İkinci tarih aralığı
plt.plot([dum2_orderdate_min, dum2_orderdate_max], [2, 2], marker='o', label='Given clients tablosunda olmayan clinet larin max ve min tarih aralıkları')
# Eksen etiketleri
plt.yticks([0, 1, 2], ['Kesişim', 'Aralık 1', 'Aralık 2'])
plt.xlabel('Tarih');
plt.title('İki Tarih Aralığının Kesişimi');
plt.legend();
plt.grid(True);
plt.show();


In [ ]:
set(app_open.client_id.unique())-set(market_order.client_id.unique())

In [ ]:
market_order[market_order.client_id.eq('6538ff945bbb65f457ca697d')]